In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar 10 22:32:28 2021

@author: frede
"""
import pygame, sys
from pygame.locals import *


class QuantumTetris():
    def __init__(self):
        self.current_state = None  # todo
        self.blocks = []  # list of block objects
        # current_block is the last element of self.blocks: self.blocks[-1]
        self.time = 0


#set color with rgb
#white, lgray, black, red, green = (255,255,255), (220,220,220) ,(0,0,0),(255,0,0), (0,255,0)
white, lgray, black, red, green = (220,220,220) , (200,200,200) ,(180,180,180) ,(255, 120, 120), (120, 255, 120)

initialresolution = (900,900)
min_resolution = (400,400)


pygame 2.0.1 (SDL 2.0.14, Python 3.8.3)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def main():
    pygame.init()
    screen = pygame.display.set_mode(initialresolution,HWSURFACE|DOUBLEBUF|RESIZABLE)
    gameDisplay = screen.copy()
    
    pygame.display.set_caption("QuanTris")
    #Size of squares
    initial_square_size = 35
    
    run = True
    
    boardHeight = 20
    boardWidth = 10
    
    main_board = Board(boardWidth, boardHeight, initial_square_size, screen.get_rect().size, "r0n0b00q0k0b01n0r0/p0p0p0p0p0p0p0p0/8/8/8/8/P0P0P0P0P0P0P0P0/R0N0B02Q0K0B03N0R0")
    run = True
    lifted = 0
    
    while run:
        gameDisplay.fill(white)
        
        mouse_RAW = pygame.mouse.get_pos()
        mouse = (mouse_RAW[0]*main_board.rescale[0], mouse_RAW[1]*main_board.rescale[1])
        
        for event in pygame.event.get():
            if event.type == pygame.VIDEORESIZE:
                main_board.resize((event.w, event.h))
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE: run = False
            elif event.type == pygame.QUIT:
                run = False
            elif event.type == MOUSEBUTTONUP:
                if event.button == 1:
                    if lifted == 2:
                        lifted = 0
                    else:
                        lifted = 1
                elif lifted == 2 and event.button == 3:
                    main_board.lifted_piece[0].direction = (main_board.lifted_piece[0].direction-1)%4
                elif lifted == 2 and event.button == 2:
                    if main_board.lifted_piece[0].type in "Bb":
                        main_board.lifted_piece[0].update_polarization((main_board.lifted_piece[0].polarization-1)%4)
            
    
        main_board.draw_board(gameDisplay)
        



        #Add a nice boarder
        pygame.draw.rect(gameDisplay,black,[main_board.hor_offset, 
                                            main_board.ver_offset,
                                            main_board.boardWidth*main_board.square_hor,
                                            main_board.boardHeight*main_board.square_ver],2)
        
        pygame.display.update()


        screen.blit(pygame.transform.scale(gameDisplay, screen.get_rect().size), (0, 0))
        pygame.display.flip()
    


In [13]:
class Board:
    def __init__(self, boardWidth, boardHeight, initial_square_size, initial_screensize, initial_state):
        self.square_size = initial_square_size
        self.boardWidth = boardWidth
        self.boardHeight = boardHeight

        # Initialized rescaling factors
        self.rescale = (1, 1)
        self.square_hor = round(initial_square_size *
                                min(self.rescale)/self.rescale[1])
        self.square_ver = round(initial_square_size *
                                min(self.rescale)/self.rescale[0])

        self.initialresolution = initial_screensize

        self.ver_offset = self.square_ver*3
        self.hor_offset = self.square_hor*8

        # Get images
        self.original_arrow = pygame.image.load(
            './Pictures/Arrow.png').convert_alpha()
        self.arrow = pygame.transform.scale(
            self.original_arrow, (2*self.square_hor, self.square_ver))
        self.arrows = [pygame.transform.rotate(
            self.arrow, -45*i) for i in range(8)]

        self.original_X = pygame.image.load(
            './Pictures/X.png').convert_alpha()
        self.Xpic = pygame.transform.scale(
            self.original_X, (2*self.square_hor, 2*self.square_ver))
        self.original_Z = pygame.image.load(
            './Pictures/Z.png').convert_alpha()
        self.Zpic = pygame.transform.scale(
            self.original_Z, (2*self.square_hor, 2*self.square_ver))

        # Black n' white
        self.black_cell = None
        self.white_cell = None

        self.rescale_tiles()
        self.cells = [[None for _ in range(boardWidth)]
                      for _ in range(boardHeight)]

        self.game_state = QuantumTetris()
        self.game_state.blocks = [[7, i*2+1, 0] for i in range(4)]+[[7, i*2+1, i*2] for i in range(4)]

    def resize(self, new_size):
        if new_size[0] < min_resolution[0] or new_size[1] < min_resolution[1]:
            pygame.display.set_mode(min_resolution, HWSURFACE | DOUBLEBUF | RESIZABLE)
            new_size = min_resolution
        else:
            pygame.display.set_mode(new_size, pygame.RESIZABLE)

        self.rescale = (new_size[0]/self.initialresolution[0],
                        new_size[1]/self.initialresolution[1])

        self.square_hor = round(
            self.square_size*min(self.rescale)/self.rescale[0])
        self.square_ver = round(
            self.square_size*min(self.rescale)/self.rescale[1])

        self.ver_offset = self.square_ver*3
        self.hor_offset = self.square_hor*8

        self.rescale_tiles()
        self.rescale_pieces()
        self.rescale_buttons()

    def rescale_tiles(self):
        # Update cell sizes
        self.black_cell = pygame.surface.Surface(
            (self.square_hor, self.square_ver))
        self.black_cell.fill(black)
        self.white_cell = pygame.surface.Surface(
            (self.square_hor, self.square_ver))
        self.white_cell.fill(white)

    def rescale_pieces(self):
        self.arrow = pygame.transform.scale(
            self.original_arrow, (2*self.square_hor, self.square_ver))

    def rescale_buttons(self):
        self.Xpic = pygame.transform.scale(
            self.original_X, (2*self.square_hor, 2*self.square_ver))
        self.Zpic = pygame.transform.scale(
            self.original_Z, (2*self.square_hor, 2*self.square_ver))

    def draw_board(self, gameDisplay):
        for H in range(self.boardHeight):
            for W in range(self.boardWidth):
                # check if current loop value is even
                if (H+W) % 2 == 0:
                    self.cells[H-1][W-1] = gameDisplay.blit(
                        self.white_cell, (self.hor_offset+self.square_hor*W, self.ver_offset + self.square_ver*H))
                else:
                    self.cells[H-1][W-1] = gameDisplay.blit(
                        self.black_cell, (self.hor_offset+self.square_hor*W, self.ver_offset + self.square_ver*H))

        self.draw_pieces(gameDisplay)
        self.draw_buttons(gameDisplay)

    def draw_pieces(self, gameDisplay):
        for block in self.game_state.blocks:  # [3,1,0], [7,5,2]
            gameDisplay.blit(self.arrows[block[2]], (self.hor_offset+self.square_hor*block[0],
                             self.ver_offset+self.square_ver*block[1], self.square_hor*self.rescale[1], self.square_ver*self.rescale[0]))

    def draw_buttons(self, gameDisplay):
        pass



In [14]:
main()

#quit from pygame & python
pygame.quit()
